In [1]:
import os
os.chdir("..")

In [2]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [3]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [4]:
Song.objects.all().delete()

(0, {})

In [31]:
import http.client

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2173464f0bmsh087f6a414d24270p11a1f5jsn1ae2008c92ce",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=20&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

In [18]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))

    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [42]:
for song in data['tracks']:
    id = str(song['artists'][0]['id'])
    print(id)
    if Artist.objects.filter(api_id=id).exists():
        print('Developer msg: data already exists in database')
        continue
    else:
        conn.request("GET", "/artists/?ids={id}".format(
                        id=id), headers=headers)
        res = conn.getresponse()
        datas = json.loads(res.read().decode('utf-8'))['artists'][0]
        # datas = datas['artists'][0]
        print(datas)
        artist = Artist(api_id=id, name = datas['name'], popularity=datas['popularity'], image=datas['images'][0]['url'], followers=datas['followers']['total'])
        artist.save()



2019zR22qK2RBvCqtudBaI
Developer msg: data already exists in database
41MozSoPIsD1dJM0CLPjZF
{'external_urls': {'spotify': 'https://open.spotify.com/artist/41MozSoPIsD1dJM0CLPjZF'}, 'followers': {'href': None, 'total': 47671206}, 'genres': ['k-pop', 'k-pop girl group', 'pop'], 'id': '41MozSoPIsD1dJM0CLPjZF', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebc9690bc711d04b3d4fd4b87c', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174c9690bc711d04b3d4fd4b87c', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178c9690bc711d04b3d4fd4b87c', 'width': 160}], 'name': 'BLACKPINK', 'popularity': 82, 'type': 'artist', 'uri': 'spotify:artist:41MozSoPIsD1dJM0CLPjZF'}
5r1tUTxVSgvBHnoDuDODPH
{'external_urls': {'spotify': 'https://open.spotify.com/artist/5r1tUTxVSgvBHnoDuDODPH'}, 'followers': {'href': None, 'total': 2088098}, 'genres': ['k-pop', 'k-pop girl group'], 'id': '5r1tUTxVSgvBHnoDuDODPH', 'images': [{'height': 6